# Overlap

In [2]:
%run -i header.py

In [8]:
def overlap(fin, fout):
    fin = pd.read_csv("quarkU/output/scr.b-scr/data/{}".format(fin))
    fout = pd.read_csv("quarkU/output/scr.b-scr/data/{}".format(fout))
    
    normIn = sp.sqrt(sp.integrate.trapz(fin["u"]**2, fin["r"]))
    normOut = sp.sqrt(sp.integrate.trapz(fout["u"]**2, fout["r"]))
    
    finterp = sp.interpolate.interp1d(fin["r"], fin["u"]/normIn)
    fouterp = sp.interpolate.interp1d(fout["r"], fout["u"]/normOut)
    
    line = fin["r"] if fin["r"].iloc[-1] < fout["r"].iloc[-1] else fout["r"]
    
    display((finterp(1), fouterp(1)))
    
    return sp.integrate.trapz(line*finterp(line)*fouterp(line), line)
    
overlap("chi_b2_2P", "yps_2S")

(array(0.46899737), array(0.21219134))

1.8146773964575502